<a href="https://colab.research.google.com/github/Dasika-Vaishnavi/NLP_John-Hewitt/blob/main/a0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 0: Tokenization!**

# **Code Submission Instructions**
1. Set `SUBMISSION_READY = True`
2. In Google Colab, please click File > Download > Download .py
3. Upload the .py file to Gradescope
# **Introduction**

This notebook implements a **byte-level tokenizer** using a **trie-based vocabulary** and an iterative **byte-pair merging algorithm** (similar to BPE).

The goal is to learn a **compact and efficient vocabulary** from raw text data, which can then be used to tokenize and encode text into integer IDs for NLP applications.

Key features:
- Works at the **byte level**, so it can handle any Unicode text without pre-tokenization.
- Uses a **maximum-length greedy tokenization** heuristic to match the longest token in the trie.
- Supports **configurable vocabulary size**, maximum token length, and control over merging across spaces.
- Can **save/load** the vocabulary for reuse in other tasks or models.

This notebook contains:
1. Class definitions for `TokenizerLearner` and `Tokenizer`.
2. Dataset loading and preprocessing.
3. Vocabulary learning loop with adjacency counting.
4. Vocabulary saving and verification.
5. Example encoding and decoding to test the tokenizer.

There is no need for a GPU for this assignment.

# **Imports and Dependencies**

In [1]:
# If you are not using Google Colab, ensure you have Python 3.8+
!pip install pygtrie

In [2]:
import pygtrie
from collections import Counter
import datasets
import json
import itertools

SUBMISSION_READY = True

# **TokenizerLearner**
The `TokenizerLearner` class builds a vocabulary using byte-pair merges.

In [3]:
class TokenizerLearner:
    def __init__(self, data_iterator, vocab_size=65536, docs_per_iter=10000, max_token_length=30, no_subwords_across_space=True):
        print(f"\nInitializing TokenizerLearner:")
        print(f"  - vocab_size: {vocab_size}")
        print(f"  - docs_per_iter: {docs_per_iter}")
        print(f"  - max_token_length: {max_token_length}")
        self.vocab_size = vocab_size
        self.data = data_iterator
        self.no_subwords_across_space = no_subwords_across_space
        self.data_iterator = iter(data_iterator)
        self.max_token_length = max_token_length
        self.docs_per_iter = docs_per_iter
        self.vocab = None

        # Byte-level vs unicode literals to control token boundaries
        self.space_char = list(b' ')[0]
        self.newline_char = list(b'\n')[0]

    def maybe_add(self, most_common_adjacencies):
        for most_common_adjacency in most_common_adjacencies:
            if self.space_char in most_common_adjacency[0][1] and self.no_subwords_across_space:
                continue
            if len(most_common_adjacency[0][0]) + len(most_common_adjacency[0][1]) > self.max_token_length:
                continue
            if (most_common_adjacency[0][0] == self.newline_char) + (most_common_adjacency[0][1] == self.newline_char) == 1:
                continue
            new_token = most_common_adjacency[0][0] + most_common_adjacency[0][1]
            new_string = bytes(new_token).decode('utf-8', errors='ignore')
            print(f"  Most common adjacency: '{new_token}' (count: {most_common_adjacency[1]})")
            print(f"  Most common adjacency: '{new_string}' (count: {most_common_adjacency[1]})")
            return new_token
        return None

    def learn(self):
        print("\nStarting vocabulary learning...")
        iteration = 0

        # Make a tokenizer
        print("  Creating tokenizer...")
        self.tokenizer = Tokenizer(vocab=self.vocab, max_token_length=self.max_token_length)

        # Initialize with all one-length byte strings
        self.tokenizer.update_trie([(x,) for x in range(256)])

        while len(self.tokenizer.trie) < self.vocab_size:
            iteration += 1
            print(f"\nIteration {iteration}:")
            print(f"Current vocab size: {len(self.tokenizer.trie)}")

            text_docs = []
            for i in range(self.docs_per_iter):
                try:
                    doc = next(self.data_iterator)
                except StopIteration:
                    self.data_iterator = iter(self.data)
                    doc = next(self.data_iterator)
                text = doc['text'].encode('utf-8')
                text_docs.append(text)

            # In this section,
            # (1) iterate through a batch of text documents, tokenizing
            # each one and counting token pair adjacences.
            # use self.tokenizer._tokenize(doc) to tokenize (so you'll need to
            # implement that first.)
            # (2) next, go through the sorted token adjacenies pair and
            # use the self.maybe_add function to get which token should be added
            # (3) update the tokenizer's trie with the new token.
            # --------------------------------- BEGIN STUDENT TODO
            # Token adjacency counting
            adj_counter = Counter()
            for text in text_docs:
                tokens = self.tokenizer._tokenize(text.decode('utf-8', errors='ignore'))
                for a, b in zip(tokens[:-1], tokens[1:]):
                    adj = ((tuple(a), tuple(b)))
                    adj_counter[adj] += 1

            # Sort adjacencies by frequency
            most_common = adj_counter.most_common()
            new_token = self.maybe_add(most_common)
            if new_token:
                self.tokenizer.update_trie([tuple(new_token)])
            else:
                print("No new valid token found. Stopping early.")
                break
            # --------------------------------- END STUDENT TODO

    def save(self, path):
        print(f"\nSaving vocabulary to {path}")
        with open(path, 'w') as f:
            for token in sorted(self.tokenizer.trie):
                f.write(json.dumps([token])+'\n')
        print(f"Saved {len(self.tokenizer.trie)} tokens")


# **Tokenizer**
The `Tokenizer` class performs encoding/decoding with the learned vocabulary.


In [4]:
class Tokenizer:
    def __init__(self, vocab_path=None, vocab=None, max_token_length=30, partial_trie=None):
        print(f"\nInitializing Tokenizer:")
        print(f"  - vocab_path: {vocab_path}")
        print(f"  - vocab size: {len(vocab) if vocab else 'None'}")
        print(f"  - max_token_length: {max_token_length}")
        self.vocab_path = vocab_path
        self.id_to_tok = []
        self.trie = pygtrie.Trie()
        self.trie = self.trie if partial_trie is None else partial_trie
        self.max_token_length = max_token_length

        if vocab_path or vocab:
            self.update_trie(vocab)

    def update_trie(self, new_vocab=None):
        print("\nUpdating trie...")
        if new_vocab is None and self.vocab_path:
            print(f"Loading from vocab file: {self.vocab_path}")
            with open(self.vocab_path, 'r') as f:
                for i, line in enumerate(f):
                    token = tuple(json.loads(line)[0])
                    self.id_to_tok.append(token)
                    self.trie[token] = i
        elif new_vocab:
            for token in new_vocab:
                print(token)
                self.id_to_tok.append(token)
                self.trie[token] = len(self.trie)

    def encode(self, text):
        return self._tokenize(text, return_ids=True)

    def decode(self, tokens):
        tokens = [self.id_to_tok[x] for x in tokens]
        return bytes(itertools.chain.from_iterable(tokens)).decode('utf-8', errors='ignore')

    def _tokenize(self, text, return_ids=False):
        # In this section,
        # (1) encode the text to receive a bytestring using
        #     text.encode('utf-8', errors='ignore')
        # (2) tokenize the string using the trie we're developing
        # As a hint, consider how to use the self.max_token_length to
        # efficiently query the trie, and note that we use the maximum-length
        # greedy tokenization heuristic.
        # (3) if return_ids=True, then return a list of integer ids. Otherwise,
        # return a list of byte lists.
        # --------------------------------- BEGIN STUDENT TODO
        if isinstance(text, bytes):
            input_bytes = text
        else:
            input_bytes = text.encode('utf-8', errors='ignore')
        tokens = []
        i = 0
        while i < len(input_bytes):
            matched = None
            max_token_len = min(self.max_token_length, len(input_bytes) - i)
            # Greedy, longest-first match
            for l in range(max_token_len, 0, -1):
                candidate = tuple(input_bytes[i:i+l])
                if candidate in self.trie:
                    matched = candidate
                    break
            if matched is not None:
                tokens.append(matched)
                i += len(matched)
            else:
                tokens.append((input_bytes[i],))
                i += 1
        if return_ids:
            return [self.trie[token] for token in tokens]
        else:
            return tokens


        # --------------------------------- END STUDENT TODO

# **Dataset Loading, Tokenizer Training, and Saving Vocabulary**
The dataset lives [here](https://huggingface.co/datasets/coms4705-hewitt/fineweb-linuxlike/tree/main). It should download automatically.

Runtime roughly scales with vocab size. Feel free to play around with it. What happens when it is less than 256?

In [5]:
print("Starting tokenizer test...")
print("Loading dataset...")
dataset = datasets.load_dataset('coms4705-hewitt/fineweb-linuxlike', 'default', streaming=True)['train']
print("Dataset loaded")

print("\nCreating TokenizerLearner...")
# learner = TokenizerLearner(dataset, vocab_size=65536, docs_per_iter=20, no_subwords_across_space=False)
learner = TokenizerLearner(dataset, vocab_size=600, docs_per_iter=20, no_subwords_across_space=True)
print("Starting learning process...")
if not SUBMISSION_READY:
    learner.learn()
print("\nLearning complete!")
learner.save('vocab-65k-fw-byte-sas.txt')

Starting tokenizer test...
Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Dataset loaded

Creating TokenizerLearner...

Initializing TokenizerLearner:
  - vocab_size: 600
  - docs_per_iter: 20
  - max_token_length: 30
Starting learning process...

Starting vocabulary learning...
  Creating tokenizer...

Initializing Tokenizer:
  - vocab_path: None
  - vocab size: None
  - max_token_length: 30

Updating trie...
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)
(24,)
(25,)
(26,)
(27,)
(28,)
(29,)
(30,)
(31,)
(32,)
(33,)
(34,)
(35,)
(36,)
(37,)
(38,)
(39,)
(40,)
(41,)
(42,)
(43,)
(44,)
(45,)
(46,)
(47,)
(48,)
(49,)
(50,)
(51,)
(52,)
(53,)
(54,)
(55,)
(56,)
(57,)
(58,)
(59,)
(60,)
(61,)
(62,)
(63,)
(64,)
(65,)
(66,)
(67,)
(68,)
(69,)
(70,)
(71,)
(72,)
(73,)
(74,)
(75,)
(76,)
(77,)
(78,)
(79,)
(80,)
(81,)
(82,)
(83,)
(84,)
(85,)
(86,)
(87,)
(88,)
(89,)
(90,)
(91,)
(92,)
(93,)
(94,)
(95,)
(96,)
(97,)
(98,)
(99,)
(100,)
(101,)
(102,)
(103,)
(104,)
(105,)
(106,)
(107,)
(108,)
(109,)


# **Testing the Tokenizer**

Once the vocabulary is learned, we can test the tokenizer by encoding some example strings and decoding them back to verify correctness.

In [6]:
examples = [
    "Hello, world!",
    "The quick brown fox jumps over the lazy dog.",
    "def tokenize(text): return text.split()",
    "🌟 Unicode characters work too! 🚀",
    "What happens when you decode a \u2603?"
]

print("Encoding and decoding examples:")
for text in examples:
    print("\nOriginal text: ", text)
    encoded = learner.tokenizer.encode(text)
    print("Encoded bytes: ", encoded)
    token_strings = [learner.tokenizer.decode([token]) for token in encoded]
    print("Individual 'decoded' token strings: ", token_strings)
    print(f"Broke {len(text)} characters into {len(encoded)} tokens!")
    decoded = learner.tokenizer.decode(encoded)
    print("Decoded text:", decoded)
    print('Is decoded same as original text?: ', decoded==text)

Encoding and decoding examples:

Original text:  Hello, world!
Encoded bytes:  [72, 566, 111, 44, 274, 263, 394, 33]
Individual 'decoded' token strings:  ['H', 'ell', 'o', ',', ' w', 'or', 'ld', '!']
Broke 13 characters into 8 tokens!
Decoded text: Hello, world!
Is decoded same as original text?:  True

Original text:  The quick brown fox jumps over the lazy dog.
Encoded bytes:  [84, 259, 32, 426, 293, 107, 287, 297, 119, 110, 270, 111, 120, 32, 106, 462, 586, 285, 358, 264, 325, 97, 122, 121, 498, 103, 46]
Individual 'decoded' token strings:  ['T', 'he', ' ', 'qu', 'ic', 'k', ' b', 'ro', 'w', 'n', ' f', 'o', 'x', ' ', 'j', 'um', 'ps', ' o', 'ver', ' the', ' l', 'a', 'z', 'y', ' do', 'g', '.']
Broke 44 characters into 27 tokens!
Decoded text: The quick brown fox jumps over the lazy dog.
Is decoded same as original text?:  True

Original text:  def tokenize(text): return text.split()
Encoded bytes:  [100, 101, 102, 286, 374, 110, 526, 101, 40, 116, 344, 116, 41, 58, 313, 116, 396, 110, 

In the cell showing the output for tokenizing the snowman emoji (☃), you see the following under **Individual 'decoded' token strings**:

```
['W', 'h', 'at', ' h', 'ap', 'pe', 'n', 's', ' wh', 'en', ' you', ' de', 'co', 'd', 'e', ' a', ' ', '', '', '', '?']
```
For the snowman emoji (right after the `' a', ' '`) you see three empty strings: `'', '', ''`.

**What do the tokens look like?**
- Each empty string corresponds to one of the bytes that make up the UTF-8 encoding of the snowman emoji (☃).
- Specifically, ☃ in UTF-8 is represented as three bytes: `0xE2`, `0x98`, `0x83` (which are 226, 152, 131 in decimal).
- Because this is a **byte-level** tokenizer and the snowman emoji doesn't appear in the limited training text, the tokenizer doesn't have a multi-byte token for it in its vocabulary.
- Instead, it falls back to splitting it into individual bytes, each as a separate token.
- When decoding these bytes to strings using `.decode('utf-8', errors='ignore')`, these non-ASCII bytes individually are invalid as standalone Unicode, so they're shown as empty strings.

**Why does this happen?**
- Your tokenizer is trained on regular text data, and with a small vocabulary size, it doesn't create tokens for rare or unseen Unicode code points like the snowman.
- The byte-level architecture guarantees every Unicode sequence is representable, even if not natively in the vocab, by breaking it into 1-byte chunks. For unseen Unicode symbols, this produces as many 1-byte tokens as needed.
- When printing, valid byte tokens are shown as their string form (like `'a'` for ASCII 97), but single bytes from a multi-byte emoji decode to empty strings because they're not valid on their own.

**Summary**:  
The snowman emoji is tokenized into three separate tokens (one for each UTF-8 byte), and each shows as an empty string because a single UTF-8 byte of an emoji doesn't decode properly as text. This demonstrates how a byte-level tokenizer gracefully handles unseen Unicode, but can't represent such tokens as readable strings unless the full sequence is present in the vocab.[1]

